In [ ]:
# 필요한 패키지/모듈 가져오기
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from collections import OrderedDict
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline

from sklearn import preprocessing
import warnings 
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [ ]:
# Google Drive Mount

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
anime = pd.read_csv('./anime.csv')
rating_complete = pd.read_csv('./rating_complete.csv', nrows = 1000000)

# Description

In [ ]:
display(anime.head())
display(rating_complete.head())

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,Producers,Licensors,Studios,Source,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min. per ep.,R - 17+ (violence & profanity),28.0,39,1251960,61971,105808,718161,71513,26678,329800,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518,273145,1174,4143,208333,1935,770,57964,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,Manga,24 min. per ep.,PG-13 - Teens 13 or older,266.0,201,558913,12944,29113,343492,25465,13925,146918,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,"TV Tokyo, Bandai Visual, Dentsu, Victor Entert...","Funimation, Bandai Entertainment",Sunrise,Original,25 min. per ep.,PG-13 - Teens 13 or older,2481.0,1467,94683,587,4300,46165,5121,5378,33719,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,"TV Tokyo, Dentsu",Unknown,Toei Animation,Manga,23 min. per ep.,PG - Children,3710.0,4369,13224,18,642,7314,766,1108,3394,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9


In [ ]:
anime.shape

(17562, 35)

In [ ]:
rating_complete.shape

(1000000, 3)

# Preprocessing

In [ ]:
cnt = rating_complete.groupby(['anime_id']).nunique()[['user_id']].reset_index()
cnt.sort_values(by = 'user_id', ascending = False, inplace = True)
cnt = cnt[:100]

In [ ]:
ll = list(cnt['anime_id'])

In [ ]:
rating_complete['ox'] = rating_complete['anime_id'].isin(ll)

In [ ]:
rating_complete['ox'].value_counts()

False    832928
True     167072
Name: ox, dtype: int64

In [ ]:
ratings = rating_complete[rating_complete['ox']==1]

In [ ]:
temp = anime[['MAL_ID','English name']]
data = pd.merge(ratings, temp, how = 'left', left_on = 'anime_id', right_on = 'MAL_ID')
data.drop('MAL_ID', axis=1,inplace=True)

In [ ]:
data.head()

,user_id,anime_id,rating,ox,English name
0,0,431,8,True,Howl's Moving Castle
1,0,121,9,True,Fullmetal Alchemist
2,0,356,9,True,Fate/stay night
3,0,1689,6,True,5 Centimeters Per Second
4,0,164,8,True,Princess Mononoke


In [ ]:
data.isnull().sum()

user_id         0
anime_id        0
rating          0
ox              0
English name    0
dtype: int64

In [ ]:
data.shape

(167072, 5)

In [ ]:
data[data['English name']=='Unknown'].shape

(0, 5)

In [ ]:
data = data[data['English name']!='Unknown']

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167072 entries, 0 to 167071
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       167072 non-null  int64 
 1   anime_id      167072 non-null  int64 
 2   rating        167072 non-null  int64 
 3   ox            167072 non-null  bool  
 4   English name  167072 non-null  object
dtypes: bool(1), int64(3), object(1)
memory usage: 6.5+ MB


In [ ]:
print("평가자수 : {}\n평가대상 애니메이션 수 : {}".format(len(data.user_id.unique()), len(data.anime_id.unique())))

평가자수 : 5419
평가대상 애니메이션 수 : 100


In [ ]:
data.to_csv("AR_CF_anime.csv", index=False)

In [ ]:
data = pd.read_csv("AR_CF_anime.csv")

# Collaborative Filtering (협업 필터링)

In [ ]:
data.head()

,user_id,anime_id,rating,ox,English name
0,0,431,8,True,Howl's Moving Castle
1,0,121,9,True,Fullmetal Alchemist
2,0,356,9,True,Fate/stay night
3,0,1689,6,True,5 Centimeters Per Second
4,0,164,8,True,Princess Mononoke


In [ ]:
data.drop(columns = 'ox', inplace = True)

In [ ]:
data[data['English name'] == "Howl's Moving Castle"]

,user_id,anime_id,rating,English name
0,0,431,8,Howl's Moving Castle
151,6,431,9,Howl's Moving Castle
164,7,431,9,Howl's Moving Castle
246,12,431,10,Howl's Moving Castle
474,19,431,10,Howl's Moving Castle
...,...,...,...,...
166596,6412,431,10,Howl's Moving Castle
166853,6418,431,9,Howl's Moving Castle
166932,6425,431,9,Howl's Moving Castle
166989,6429,431,9,Howl's Moving Castle


In [ ]:
df = data.pivot_table('rating', index = 'user_id', columns = 'English name')
df.head()

English name,5 Centimeters Per Second,A Silent Voice,Akame ga Kill!,Angel Beats!,Another,Assassination Classroom,Attack on Titan,Attack on Titan Season 2,Attack on Titan Season 3,Attack on Titan Season 3 Part 2,BTOOOM!,Bakemonogatari,Beyond the Boundary,Black Lagoon,Blue Exorcist,Charlotte,Clannad,Clannad ~After Story~,Code Geass:Lelouch of the Rebellion,Code Geass:Lelouch of the Rebellion R2,Cowboy Bebop,Darker than Black,Deadman Wonderland,Death Note,Death Parade,Demon Slayer:Kimetsu no Yaiba,Dragon Ball Z,Durarara!!,ERASED,Elfen Lied,Fairy Tail,Fate/Zero,Fate/Zero Season 2,Fate/stay night,Food Wars! Shokugeki no Soma,Food Wars! The Second Plate,Fullmetal Alchemist,Fullmetal Alchemist:Brotherhood,Guilty Crown,Gurren Lagann,Haganai:I don't have many friends,Haikyu!!,High School DxD,High School of the Dead,Howl's Moving Castle,Hunter x Hunter,Hyouka,Is It Wrong to Try to Pick Up Girls in a Dungeon?,K-ON!,KILL la KILL,KonoSuba:God's Blessing on This Wonderful World!,KonoSuba:God's Blessing on This Wonderful World! 2,Kuroko's Basketball,Log Horizon,"Love, Chunibyo & Other Delusions!",Maid Sama!,Mob Psycho 100,My Hero Academia,My Hero Academia 2,My Hero Academia 3,My Little Monster,My Neighbor Totoro,My Teen Romantic Comedy SNAFU,My Teen Romantic Comedy SNAFU TOO!,Naruto,Neon Genesis Evangelion,Nisekoi:False Love,"No Game, No Life",Noragami Aragoto,Noragami:Stray God,One Punch Man,OreImo,Ouran High School Host Club,Overlord,Parasyte -the maxim-,Princess Mononoke,Psycho-Pass,Puella Magi Madoka Magica,Re:ZERO -Starting Life in Another World-,Samurai Champloo,Seraph of the End:Vampire Reign,Soul Eater,Spirited Away,Steins;Gate,Sword Art Online,Sword Art Online II,Terror in Resonance,The Devil is a Part-Timer!,The Familiar of Zero,The Future Diary,The Girl Who Leapt Through Time,The Melancholy of Haruhi Suzumiya,The Pet Girl of Sakurasou,The Seven Deadly Sins,Tokyo Ghoul,Tokyo Ghoul √A,Toradora!,Your Lie in April,Your Name.,anohana:The Flower We Saw That Day
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,9.0,7.0,6.0,NaN,NaN,10.0,10.0,10.0,NaN,NaN,8.0,NaN,NaN,8.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,9.0,NaN,9.0,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,8.0,8.0,NaN,10.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,6.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,9.0,8.0,8.0,NaN,NaN,8.0,8.0,10.0,NaN,NaN,8.0,NaN,NaN,8.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,10.0,10.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,9.0,9.0,10.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,10.0,NaN,NaN,NaN,10.0,8.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,8.0,9.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,9.0,8.0,NaN,NaN,NaN,NaN,NaN
3,8.0,NaN,7.0,8.0,8.0,NaN,NaN,NaN,NaN,NaN,8.0,8.0,8.0,9.0,8.0,8.0,8.0,NaN,8.0,8.0,9.0,7.0,7.0,NaN,7.0,NaN,NaN,8.0,7.0,9.0,NaN,8.0,9.0,NaN,8.0,8.0,NaN,NaN,NaN,NaN,7.0,NaN,7.0,7.0,NaN,NaN,8.0,NaN,NaN,NaN,9.0,8.0,NaN,NaN,8.0,8.0,NaN,NaN,NaN,NaN,9.0,NaN,8.0,8.0,NaN,8.0,8.0,8.0,8.0,8.0,8.0,7.0,NaN,8.0,8.0,NaN,9.0,NaN,9.0,9.0,7.0,NaN,NaN,9.0,NaN,NaN,NaN,8.0,7.0,8.0,NaN,NaN,8.0,7.0,NaN,NaN,8.0,8.0,9.0,8.0
4,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
df.shape

(5419, 100)

- user들별로 평점을 주는 범위가 다르기 때문에 scaling하여 조정

In [ ]:
df_scaled = df.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

## Item-based collaborative filtering

In [ ]:
df_scaled_i = df_scaled.transpose().fillna(0)
df_scaled_i.head(5)

user_id,0,1,2,3,4,5,6,7,8,11,12,13,14,16,17,18,19,20,21,22,23,25,27,28,29,30,31,32,33,34,35,36,37,38,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,61,62,63,64,65,66,67,68,70,71,72,73,74,75,77,78,79,80,81,82,83,84,85,87,88,89,90,91,93,94,95,97,99,101,102,103,104,107,108,110,111,112,113,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,133,134,135,136,137,138,139,143,145,146,147,148,149,150,152,153,154,155,156,157,158,160,162,163,164,165,166,167,169,170,171,172,173,174,177,178,179,181,184,185,186,187,188,189,190,192,193,194,195,196,197,198,201,202,204,205,207,208,209,210,211,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,231,232,235,237,238,239,240,241,243,244,246,247,248,249,251,252,253,254,255,257,258,259,260,262,263,264,265,266,267,268,269,270,272,273,274,275,276,277,278,281,282,283,284,285,286,287,288,289,290,291,292,293,294,296,297,298,299,300,301,302,304,...,6137,6138,6139,6140,6141,6143,6144,6145,6146,6147,6148,6149,6151,6152,6153,6154,6155,6156,6157,6158,6159,6160,6161,6162,6163,6165,6166,6168,6169,6170,6171,6172,6173,6174,6175,6176,6177,6178,6180,6181,6182,6183,6184,6185,6186,6187,6188,6189,6190,6191,6195,6197,6198,6199,6200,6201,6202,6204,6206,6207,6208,6209,6210,6211,6212,6213,6214,6215,6216,6218,6219,6220,6221,6222,6223,6224,6225,6227,6228,6229,6232,6234,6235,6237,6238,6239,6240,6241,6242,6243,6244,6245,6246,6247,6248,6249,6251,6252,6253,6254,6255,6256,6257,6258,6259,6260,6261,6262,6263,6264,6266,6267,6268,6269,6270,6271,6272,6273,6274,6275,6276,6277,6278,6279,6280,6282,6283,6284,6285,6286,6287,6288,6289,6290,6291,6293,6297,6298,6299,6300,6301,6302,6303,6305,6308,6309,6310,6311,6312,6313,6314,6316,6317,6318,6319,6320,6321,6322,6323,6324,6325,6327,6328,6329,6330,6331,6332,6334,6335,6337,6338,6339,6341,6342,6343,6344,6345,6346,6347,6348,6349,6350,6351,6352,6354,6355,6356,6357,6360,6361,6364,6365,6366,6367,6368,6370,6371,6373,6374,6375,6376,6378,6381,6382,6383,6384,6385,6386,6387,6388,6389,6390,6392,6393,6394,6395,6396,6397,6398,6399,6402,6403,6404,6405,6406,6407,6408,6410,6411,6412,6413,6414,6415,6416,6417,6418,6419,6420,6421,6422,6423,6425,6426,6428,6429,6430,6431,6432,6433,6434
English name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5 Centimeters Per Second,-0.571429,0.000000,0.0,0.008621,0.366667,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.091463,-0.372685,0.000000,-0.119481,0.0,0.00,0.0,0.0,0.0,-0.229167,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.078818,0.0,0.0,-0.483871,0.000000,-0.75,0.0,0.000000,-0.238095,-0.009146,0.000000,-0.909091,0.000000,0.000000,0.0,-0.371795,-0.134921,-0.075758,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,-0.100775,0.0,0.000000,0.0,0.000,-0.034483,0.000000,-0.337838,0.1,0.0,0.0,0.205556,-0.222222,-0.561111,0.20625,0.0,0.426471,0.0,0.0,0.000000,0.080460,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.2,0.079832,0.000000,0.0,0.000000,0.13,0.124294,-0.173469,0.000000,0.000000,-0.572327,0.0,-0.10119,0.0,0.493902,0.0,0.0,0.000000,0.0,0.000000,0.146667,0.0,-0.24569,0.0,0.096154,0.000000,0.0,0.000,0.000000,0.0,0.0,0.0,0.000,0.072917,0.067708,0.314815,0.000000,0.0,0.0,0.000000,-0.3,0.000000,0.258537,0.137255,-0.14375,-0.087662,0.000000,0.000000,0.00,0.34375,0.000000,0.00,-0.288889,0.000000,0.0,0.00,0.000000,0.000000,0.282313,0.000000,0.000000,-0.266917,0.000000,0.355932,0.000000,0.000000,0.000000,0.0,0.0,0.0,-0.176471,-0.097015,0.0,0.0,0.000000,0.000000,-0.374468,0.000000,-0.258621,0.000000,0.406504,-0.708333,0.00,-0.463675,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.

In [ ]:
movie_sim_i = cosine_similarity(df_scaled_i, df_scaled_i)
print(movie_sim_i.shape)

(100, 100)


In [ ]:
movie_sim_i = pd.DataFrame(movie_sim_i, index = df_scaled_i.index, columns = df_scaled_i.index)

In [ ]:
movie_sim_i.head()

English name,5 Centimeters Per Second,A Silent Voice,Akame ga Kill!,Angel Beats!,Another,Assassination Classroom,Attack on Titan,Attack on Titan Season 2,Attack on Titan Season 3,Attack on Titan Season 3 Part 2,BTOOOM!,Bakemonogatari,Beyond the Boundary,Black Lagoon,Blue Exorcist,Charlotte,Clannad,Clannad ~After Story~,Code Geass:Lelouch of the Rebellion,Code Geass:Lelouch of the Rebellion R2,Cowboy Bebop,Darker than Black,Deadman Wonderland,Death Note,Death Parade,Demon Slayer:Kimetsu no Yaiba,Dragon Ball Z,Durarara!!,ERASED,Elfen Lied,Fairy Tail,Fate/Zero,Fate/Zero Season 2,Fate/stay night,Food Wars! Shokugeki no Soma,Food Wars! The Second Plate,Fullmetal Alchemist,Fullmetal Alchemist:Brotherhood,Guilty Crown,Gurren Lagann,Haganai:I don't have many friends,Haikyu!!,High School DxD,High School of the Dead,Howl's Moving Castle,Hunter x Hunter,Hyouka,Is It Wrong to Try to Pick Up Girls in a Dungeon?,K-ON!,KILL la KILL,KonoSuba:God's Blessing on This Wonderful World!,KonoSuba:God's Blessing on This Wonderful World! 2,Kuroko's Basketball,Log Horizon,"Love, Chunibyo & Other Delusions!",Maid Sama!,Mob Psycho 100,My Hero Academia,My Hero Academia 2,My Hero Academia 3,My Little Monster,My Neighbor Totoro,My Teen Romantic Comedy SNAFU,My Teen Romantic Comedy SNAFU TOO!,Naruto,Neon Genesis Evangelion,Nisekoi:False Love,"No Game, No Life",Noragami Aragoto,Noragami:Stray God,One Punch Man,OreImo,Ouran High School Host Club,Overlord,Parasyte -the maxim-,Princess Mononoke,Psycho-Pass,Puella Magi Madoka Magica,Re:ZERO -Starting Life in Another World-,Samurai Champloo,Seraph of the End:Vampire Reign,Soul Eater,Spirited Away,Steins;Gate,Sword Art Online,Sword Art Online II,Terror in Resonance,The Devil is a Part-Timer!,The Familiar of Zero,The Future Diary,The Girl Who Leapt Through Time,The Melancholy of Haruhi Suzumiya,The Pet Girl of Sakurasou,The Seven Deadly Sins,Tokyo Ghoul,Tokyo Ghoul √A,Toradora!,Your Lie in April,Your Name.,anohana:The Flower We Saw That Day
English name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5 Centimeters Per Second,1.000000,-0.067031,0.018133,-0.005227,0.021644,-0.010705,-0.093742,-0.037877,-0.055865,-0.081783,0.014035,0.011849,0.014817,-0.009643,0.024491,-0.022828,0.005129,-0.047885,-0.098570,-0.092621,0.019218,-0.003167,0.036679,-0.080126,0.017962,-0.063972,0.011085,-0.014446,-0.063247,0.036780,-0.013151,-0.014524,-0.020816,0.032082,-0.033547,-0.031959,-0.018234,-0.101049,0.015000,-0.000782,0.029494,-0.055433,0.015485,0.033848,-0.038529,-0.065594,0.038003,0.028923,0.045634,-0.000171,0.006249,-0.034862,-0.028888,-0.019864,0.009207,-0.011590,-0.014357,-0.019083,-0.062552,-0.051915,0.021382,0.073720,0.010236,0.018374,-0.001010,0.050152,0.016004,-0.045904,-0.028037,-0.003150,-0.048580,0.033263,-0.036277,0.000517,-0.030557,-0.026014,-0.063611,-0.000598,-0.046605,-0.010702,0.038692,-0.013455,-0.036511,-0.090927,-0.012753,0.032637,-0.003492,0.053860,0.027792,0.002314,0.041680,0.047635,-0.041032,-0.011633,-0.010743,0.043293,0.002310,-0.050208,-0.104348,-0.002338
A Silent Voice,-0.067031,1.000000,-0.197779,0.021218,-0.163015,-0.049679,0.096454,0.109050,0.147090,0.274532,-0.164660,0.073292,-0.133008,-0.030817,-0.172121,-0.140068,-0.035306,0.193146,0.127005,0.166173,0.065663,-0.000428,-0.179443,0.121290,0.022493,0.153025,-0.033398,0.010116,0.079948,-0.100114,-0.100635,0.061196,0.129969,-0.120014,0.052279,0.022196,-0.015451,0.247078,-0.111882,0.124561,-0.215940,0.099737,-0.180837,-0.179223,0.071989,0.231816,0.004394,-0.191058,-0.085511,0.000681,-0.027878,-0.001398,-0.035819,-0.082083,-0.116987,-0.045217,0.090753,-0.035534,0.069682,0.065088,-0.133324,0.001892,-0.045582,0.020564,-0.056276,0.035750,-0.171624,0.026199,0.025381,-0.079448,0.119553,-0.186812,-0.031384,-0.092212,0.070494,0.091300,0.062060,0.061048,0.083811,0.050159,-0.184629,-0.068861,0.125748,0.244795,-0.213298,-0.276000,0.001939,-0.119381,-0.121398,-0.159968,-0.003655,-0.050759,0.027437,-0.130281,-0.1351

In [ ]:
movie_sim_i["Howl's Moving Castle"].sort_values(ascending=False)[:10]

English name
Howl's Moving Castle               1.000000
Spirited Away                      0.306950
Princess Mononoke                  0.217722
My Neighbor Totoro                 0.127003
Fullmetal Alchemist:Brotherhood    0.111779
Your Name.                         0.075504
Hunter x Hunter                    0.075342
A Silent Voice                     0.071989
Steins;Gate                        0.067345
Cowboy Bebop                       0.056344
Name: Howl's Moving Castle, dtype: float64

## User-based Collaborative Filtering

In [ ]:
df_scaled_u = df_scaled.fillna(0)
df_scaled_u.head(5)

English name,5 Centimeters Per Second,A Silent Voice,Akame ga Kill!,Angel Beats!,Another,Assassination Classroom,Attack on Titan,Attack on Titan Season 2,Attack on Titan Season 3,Attack on Titan Season 3 Part 2,BTOOOM!,Bakemonogatari,Beyond the Boundary,Black Lagoon,Blue Exorcist,Charlotte,Clannad,Clannad ~After Story~,Code Geass:Lelouch of the Rebellion,Code Geass:Lelouch of the Rebellion R2,Cowboy Bebop,Darker than Black,Deadman Wonderland,Death Note,Death Parade,Demon Slayer:Kimetsu no Yaiba,Dragon Ball Z,Durarara!!,ERASED,Elfen Lied,Fairy Tail,Fate/Zero,Fate/Zero Season 2,Fate/stay night,Food Wars! Shokugeki no Soma,Food Wars! The Second Plate,Fullmetal Alchemist,Fullmetal Alchemist:Brotherhood,Guilty Crown,Gurren Lagann,Haganai:I don't have many friends,Haikyu!!,High School DxD,High School of the Dead,Howl's Moving Castle,Hunter x Hunter,Hyouka,Is It Wrong to Try to Pick Up Girls in a Dungeon?,K-ON!,KILL la KILL,KonoSuba:God's Blessing on This Wonderful World!,KonoSuba:God's Blessing on This Wonderful World! 2,Kuroko's Basketball,Log Horizon,"Love, Chunibyo & Other Delusions!",Maid Sama!,Mob Psycho 100,My Hero Academia,My Hero Academia 2,My Hero Academia 3,My Little Monster,My Neighbor Totoro,My Teen Romantic Comedy SNAFU,My Teen Romantic Comedy SNAFU TOO!,Naruto,Neon Genesis Evangelion,Nisekoi:False Love,"No Game, No Life",Noragami Aragoto,Noragami:Stray God,One Punch Man,OreImo,Ouran High School Host Club,Overlord,Parasyte -the maxim-,Princess Mononoke,Psycho-Pass,Puella Magi Madoka Magica,Re:ZERO -Starting Life in Another World-,Samurai Champloo,Seraph of the End:Vampire Reign,Soul Eater,Spirited Away,Steins;Gate,Sword Art Online,Sword Art Online II,Terror in Resonance,The Devil is a Part-Timer!,The Familiar of Zero,The Future Diary,The Girl Who Leapt Through Time,The Melancholy of Haruhi Suzumiya,The Pet Girl of Sakurasou,The Seven Deadly Sins,Tokyo Ghoul,Tokyo Ghoul √A,Toradora!,Your Lie in April,Your Name.,anohana:The Flower We Saw That Day
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-0.571429,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.178571,0.000000,0.000000,0.178571,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,-0.071429,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.071429,0.000000,0.0,0.000000,0.000000,0.000000,0.0,-0.071429,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.428571,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.138462,-0.261538,-0.461538,0.000000,0.0,0.338462,0.338462,0.338462,0.0,0.000000,-0.061538,0.000000,0.000000,-0.061538,0.000000,0.000000,0.0,0.338462,0.000000,0.000000,0.000000,0.000000,0.138462,0.000000,0.138462,0.0,0.000000,0.138462,0.000000,0.0,0.000000,0.000000,0.000000,-0.061538,-0.061538,0.000000,0.338462,0.0,0.0,0.000000,-0.061538,0.000000,0.000000,0.000000,0.138462,0.000000,0.0,0.0,-0.461538,-0.261538,-0.261538,-0.261538,0.0,0.000000,0.000000,0.000000,0.138462,-0.061538,-0.061538,0.000000,0.0,-0.061538,-0.061538,0.338462,0.000000,0.000000,-0.061538,0.000000,0.000000,-0.061538,0.000000,0.000000,0.000000,0.138462,0.000000,0.000000,0.0,-0.061538,0.000000,0.000000,0.0,0.338462,0.338462,-0.661538,-0.661538,0.0,0.000000,0.000000,0.000000,0.000000,0.0,-0.261538,0.000000,0.000000,0.0,0.138462,0.138462,0.338462,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.078947,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.078947,0.328947,0.000000,0.000000,0.000000,0.328947,-0.171053,0.000000,0.0,0.000000,0.

In [ ]:
movie_sim_u = cosine_similarity(df_scaled_u, df_scaled_u)
print(movie_sim_u.shape)

(5419, 5419)


In [ ]:
movie_sim_u = pd.DataFrame(movie_sim_u, index = df_scaled_u.index, columns = df_scaled_u.index)

In [ ]:
movie_sim_u.head()

user_id,0,1,2,3,4,5,6,7,8,11,12,13,14,16,17,18,19,20,21,22,23,25,27,28,29,30,31,32,33,34,35,36,37,38,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,61,62,63,64,65,66,67,68,70,71,72,73,74,75,77,78,79,80,81,82,83,84,85,87,88,89,90,91,93,94,95,97,99,101,102,103,104,107,108,110,111,112,113,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,133,134,135,136,137,138,139,143,145,146,147,148,149,150,152,153,154,155,156,157,158,160,162,163,164,165,166,167,169,170,171,172,173,174,177,178,179,181,184,185,186,187,188,189,190,192,193,194,195,196,197,198,201,202,204,205,207,208,209,210,211,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,231,232,235,237,238,239,240,241,243,244,246,247,248,249,251,252,253,254,255,257,258,259,260,262,263,264,265,266,267,268,269,270,272,273,274,275,276,277,278,281,282,283,284,285,286,287,288,289,290,291,292,293,294,296,297,298,299,300,301,302,304,...,6137,6138,6139,6140,6141,6143,6144,6145,6146,6147,6148,6149,6151,6152,6153,6154,6155,6156,6157,6158,6159,6160,6161,6162,6163,6165,6166,6168,6169,6170,6171,6172,6173,6174,6175,6176,6177,6178,6180,6181,6182,6183,6184,6185,6186,6187,6188,6189,6190,6191,6195,6197,6198,6199,6200,6201,6202,6204,6206,6207,6208,6209,6210,6211,6212,6213,6214,6215,6216,6218,6219,6220,6221,6222,6223,6224,6225,6227,6228,6229,6232,6234,6235,6237,6238,6239,6240,6241,6242,6243,6244,6245,6246,6247,6248,6249,6251,6252,6253,6254,6255,6256,6257,6258,6259,6260,6261,6262,6263,6264,6266,6267,6268,6269,6270,6271,6272,6273,6274,6275,6276,6277,6278,6279,6280,6282,6283,6284,6285,6286,6287,6288,6289,6290,6291,6293,6297,6298,6299,6300,6301,6302,6303,6305,6308,6309,6310,6311,6312,6313,6314,6316,6317,6318,6319,6320,6321,6322,6323,6324,6325,6327,6328,6329,6330,6331,6332,6334,6335,6337,6338,6339,6341,6342,6343,6344,6345,6346,6347,6348,6349,6350,6351,6352,6354,6355,6356,6357,6360,6361,6364,6365,6366,6367,6368,6370,6371,6373,6374,6375,6376,6378,6381,6382,6383,6384,6385,6386,6387,6388,6389,6390,6392,6393,6394,6395,6396,6397,6398,6399,6402,6403,6404,6405,6406,6407,6408,6410,6411,6412,6413,6414,6415,6416,6417,6418,6419,6420,6421,6422,6423,6425,6426,6428,6429,6430,6431,6432,6433,6434
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.000000,-0.018518,0.000000,-0.002677,-0.257880,0.037987,0.055806,-0.041720,0.000000,0.041976,0.0,0.000000e+00,-0.003798,-0.029060,0.103556,0.003661,0.058735,0.000000,0.006217,0.050649,0.000000,0.000000,-0.032809,0.062032,0.000000,-0.034783,0.0,-0.012480,0.034121,-0.030039,0.0,-0.002815,-0.050965,0.000000,0.160597,0.023277,0.322396,0.000000,0.005899,-0.166821,-0.069704,-0.020706,0.709746,0.007466,-0.032425,0.058729,0.089735,0.017965,-0.012421,-0.034975,0.000000,-0.144626,-0.067613,-0.038850,-0.089175,-0.003403,0.007098,-0.036491,0.000000,-0.172200,0.041740,0.0,-0.039335,0.0,-0.008341,0.004510,0.000000,0.177749,-0.060279,0.008742,0.000000,-0.085442,0.101845,0.083703,-0.056930,-0.007864,-0.138878,0.000000e+00,0.043574,-0.013719,-0.166037,0.065795,-0.065795,0.045274,0.000000,0.000000,0.000000,-0.013430,-0.059804,0.000000,0.0,0.005195,-0.102957,-0.045148,0.082175,-0.039848,0.000000,0.218719,0.000000,0.063543,0.0,-0.296338,0.000000,-0.153552,0.044755,0.042259,0.000000,-0.073034,-0.070011,0.053332,0.0,-0.058714,-0.188526,-0.002241,-0.056235,0.009985,-0.019077,0.000000,0.000000,0.000000,-0.098651,-0.074198,-0.104326,0.016787,0.000000,-0.062075,0.000000,-0.002193,0.003401,-0.121729,0.009533,0.066921,0.108843,0.000609,-0.020021,-0.039477,-0.179278,0.004550,0.000000,0.129935,0.073715,0.0,0.131722,-0.134526,-0.00

In [ ]:
movie_sim_u[0].sort_values(ascending=False)[:11][0]

1.0000000000000002

In [ ]:
sum(movie_sim_u[0].sort_values(ascending=False))

10.150682591520182

In [ ]:
movie_sim_u[0].sort_values(ascending=False)[1:11]

user_id
2057    0.835711
48      0.709746
3959    0.686792
1842    0.643498
3796    0.639655
2618    0.607751
3740    0.597710
3494    0.575708
2565    0.569372
1303    0.561939
Name: 0, dtype: float64

In [ ]:
top10_user = movie_sim_u[0].sort_values(ascending=False)[1:11].keys()

In [ ]:
def weighted_score(df, target_user, k, similarity, movie_name):
  weighted_score = 0
  score_sum = 0
  sim_list = similarity[target_user].sort_values(ascending = False)[1:k+1]
  neighbors = sim_list.keys()
  scores = df[movie_name]
  for i in neighbors:
    score_sum += scores[i] * sim_list[i]
  weighted_score = score_sum / sum(sim_list)
 
  return weighted_score

In [ ]:
weighted_score(df_scaled_u, 0, 10, movie_sim_u, 'Death Note')

0.07533318950309488

In [ ]:
movies = df_scaled_u.columns.tolist()
weighted_score_dict = dict()
user0_rating = df_scaled_u.iloc[0, :]
for movie in movies:
  if user0_rating[movie] == 0:
    weighted_score_dict[movie] = weighted_score(df_scaled_u, 0, 10, movie_sim_u, movie)

In [ ]:
weighted_score_dict

{'A Silent Voice': 0.0,
 'Akame ga Kill!': 0.0,
 'Angel Beats!': 0.0,
 'Another': 0.0,
 'Assassination Classroom': 0.0,
 'Attack on Titan': 0.010037883517196797,
 'Attack on Titan Season 2': 0.0,
 'Attack on Titan Season 3': 0.0,
 'Attack on Titan Season 3 Part 2': 0.0,
 'BTOOOM!': 0.0,
 'Bakemonogatari': 0.0,
 'Beyond the Boundary': 0.0,
 'Black Lagoon': 0.010059285134035235,
 'Blue Exorcist': 0.02217984888834131,
 'Charlotte': 0.0,
 'Clannad': 0.007826492583108037,
 'Clannad ~After Story~': 0.02772901098990695,
 'Code Geass:Lelouch of the Rebellion': 0.05872755382719686,
 'Code Geass:Lelouch of the Rebellion R2': 0.08375514828371697,
 'Cowboy Bebop': 0.012677467213430322,
 'Darker than Black': 0.0,
 'Deadman Wonderland': -0.009713572296915616,
 'Death Note': 0.07533318950309488,
 'Death Parade': -0.016169083808268755,
 'Demon Slayer:Kimetsu no Yaiba': 0.0,
 'Dragon Ball Z': 0.010037883517196797,
 'Durarara!!': 0.05065318647547402,
 'ERASED': -0.003515018219188868,
 'Elfen Lied': 0.03

In [ ]:
weighted_score_dict_ = sorted(weighted_score_dict.items(), key = lambda item: item[1], reverse = True)
weighted_score_dict_

[('Code Geass:Lelouch of the Rebellion R2', 0.08375514828371697),
 ('Death Note', 0.07533318950309488),
 ('Code Geass:Lelouch of the Rebellion', 0.05872755382719686),
 ('Durarara!!', 0.05065318647547402),
 ('Toradora!', 0.05012005050025288),
 ('High School of the Dead', 0.045791054032049486),
 ('Your Lie in April', 0.03874459999960932),
 ('Elfen Lied', 0.035532475474071984),
 ('Steins;Gate', 0.03393507833011542),
 ('High School DxD', 0.027983421908474682),
 ('Clannad ~After Story~', 0.02772901098990695),
 ('Fullmetal Alchemist:Brotherhood', 0.025213139002842003),
 ('Psycho-Pass', 0.022391039510345937),
 ('Blue Exorcist', 0.02217984888834131),
 ("KonoSuba:God's Blessing on This Wonderful World! 2", 0.02179311295897091),
 ('My Hero Academia 2', 0.02179311295897091),
 ('Cowboy Bebop', 0.012677467213430322),
 ('Black Lagoon', 0.010059285134035235),
 ('Attack on Titan', 0.010037883517196797),
 ('Dragon Ball Z', 0.010037883517196797),
 ('KILL la KILL', 0.00913904736989102),
 ("KonoSuba:God's

In [ ]:
pd.DataFrame(weighted_score_dict_, columns = ['English name', 'weighted score'])[:10]

,English name,weighted score
0,Code Geass:Lelouch of the Rebellion R2,0.083755
1,Death Note,0.075333
2,Code Geass:Lelouch of the Rebellion,0.058728
3,Durarara!!,0.050653
4,Toradora!,0.050120
5,High School of the Dead,0.045791
6,Your Lie in April,0.038745
7,Elfen Lied,0.035532
8,Steins;Gate,0.033935
9,High School DxD,0.027983
